## Saving `dysh` data to external files
This notebook gives examples of how to write out selected data from [`GBTFitsLoad`](https://dysh.readthedocs.io/en/latest/modules/dysh.fits.html#module-dysh.fits.gbtfitsload) and how to save 
[`Spectrum`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#module-dysh.spectra.spectrum), 
[`Scan`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#module-dysh.spectra.scan), and 
[`ScanBlock`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock) to different formats.

In [1]:
import os
import wget
import astropy.units as u
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.spectra.spectrum import Spectrum
from dysh.spectra.scan import ScanBlock
import matplotlib.pyplot as plt
import numpy as np

### Let's start with `GBTFITSLoad`
First read  in some data

In [2]:
filename = "TGBT21A_501_11.raw.vegas.fits"
if not os.path.isfile(filename):
    url = f"http://www.gb.nrao.edu/dysh/example_data/onoff-L/data/{filename}"
    print(f"Downloading {filename}")
    wget.download(url,out=filename)
    print(f"\nRetrieved {filename}")
else:
    print(f"{filename} already downloaded")

TGBT21A_501_11.raw.vegas.fits already downloaded


In [3]:
sdfits = GBTFITSLoad(filename)
sdfits.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,152,NGC2415,3784.0,OnOff,1,1.617185,1.420406,5,2,151,1,286.218008,41.62843
1,153,NGC2415,3784.0,OnOff,2,1.617185,1.420406,5,2,151,1,286.886521,41.118134


### Calibrate a position switched scan.  This returns a `ScanBlock` containing one [`PSScan`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.scan.PSScan) with 151 ONs and 151 OFFs.

In [ ]:
psscan = sdfits.getps(scan=152, ifnum=0, plnum=0)

In [ ]:
print(f"Number of integrations = {psscan[0].nrows}")

### Get the time average of the calibrated data
This method returns a `Spectrum`.

In [ ]:
ta = psscan.timeaverage()

In [ ]:
ta.plot()

---

# 1.  Reading and Writing Individual Spectra

###  ASCII output
`dysh` supports output to text files in a variety of [formats familiar to users of astropy](https://docs.astropy.org/en/stable/io/ascii/index.html#id1):
* basic
* commented_header
* ECSV
* fixed_width
* IPAC
* MRT
* votable

In [ ]:
fmt = [ 
    "basic",
    "commented_header",
    "ecsv",
    "fixed_width",
    "ipac",
    "mrt",
    "votable",
]
for f in fmt:
    file = f"testwrite.{f}"
    ta.write(file, format=f, overwrite=True)


### We can also write a spectrum to FITS format

In [ ]:
ta.write("testwrite.fits",format="fits",overwrite=True)

### We can read spectra in FITS and a few formats
[As noted in astropy, ECSV ](https://docs.astropy.org/en/stable/io/ascii/ecsv.html#ecsv-format) is the only ASCII format that can make a lossless output-input roundtrip and thus reproduce an original spectrum.

In [ ]:
s1=Spectrum.read("testwrite.fits",format="fits")
s1.plot()

In [ ]:
s2=Spectrum.read("testwrite.ecsv",format="ecsv")
s2.plot()

### GBTIDL ASCII format
`dysh` can read text files created by GBTIDL's `write_ascii` function. However, those files do not provide sufficient metadata to to fully recreate the spectrum.  (For instance, they do not have complete sky coordinate information).  

In [ ]:
s3 = Spectrum.read("onoff-L_gettp_156_intnum_0_HEL.ascii", format='gbtidl')

In [ ]:
print(s3,"\n",s3.meta)

### `dysh` can even read compressed ASCII files
Note these data have velocity on the spectral axis

In [ ]:
s4 = Spectrum.read("onoff-L_getps_152_RADI-HEL.ascii.gz", format='gbtidl')

In [ ]:
print(s4,"\n",s4.meta)

#### Plot it with simple matplotlib commands

In [ ]:
plt.xlabel(f"Velocity ({s4.meta['VELDEF']}) [{s4.spectral_axis.unit}]")
plt.ylabel(f"$T_A$ [{s4.flux.unit}]")
plt.plot(s4.spectral_axis,s4.flux)

---

# 2. Writing Calibrated Data to SDFITS
You can write the calibrated data from a ScanBlock in SDFITS format.  If there are multiple Scans in the ScanBlock, they will all be written to the same binary table (useful for gbtgridder)

In [ ]:
psscan.write('scanblock.fits',overwrite=True)

---

# 3. Writing out selected data from `GBTFITSLoad`
The `write()` method of `GBTFITSLoad` supports down-selection of data. Data can be selected on any SDFITS column.

In [ ]:
sdfits.write("mydata.fits",plnum=1,ifnum=[0,2],intnum=np.arange(100),overwrite=True)

### These data, can of course, be read back in

In [ ]:
sdfits2 = GBTFITSLoad("mydata.fits")
sdfits2.summary()

### Writing SDFITS to multiple files 
If the data came from multiple files, for instance VEGAS banks, then by default they are written to multiple files, so
`sdfits.write('mydata.fits') `
would write to mydata0.fits, mydata1.fits, ... mydataN.fits.
A future enhancement will preserve the A,B,C alphabetic extensions of the input data.